In [ ]:
from IPython.display import display, Math
from scipy.optimize import brute, fmin
import numpy as np
import pandas as pd
from scipy.integrate import quad
from typing import Callable, Union, Self
from numpy.typing import NDArray


def render_latex(expression):
    display(Math(expression))

1. δ predicts an option's change relative to a change in underlying movement. Call Options have positive deltas, since they move in the same direction as stock price. This is actually the mysterious N(d1). Remember N(d2)? Put Options have negative deltas as they move in opposite direction as S
2. What does a delta of +/- 1 mean?

1. γ. Do you think δ stays constant? What would happen if it did?
2. γ, therefore, represents the change in deltas. In other words, how an option's δ and NOT price changes with respect to the underlying dynamics.
3. Think of gamma as?? (clue: what happens if delta changes or doesn't change?)

Gamma reflects changes in option's delta. So, essentially, it captures the probability of option expiring in the money as stock price changes. Think of what delta represents again and what a higher delta means.

θ (think of time). represents the change in options price (decrease actually. Why?) with respect to time, assuming no change in underlying or volatility).

𝓥 represents the change in option price with respect to change in implied volatility.

ρ represents the change in the option value with respect to change in interest rate.

# **In basic math:**

**Call and Put**:

1. delta (Call) = dC /dS = N(d1)
2. delta(Put) = dP/dS = N(d1) - 1  
3. delta(Call) = d^2V/dS^2 = N'(d1)/S*(sigma)*(sqrt(dt))
4. delta(Put) = d^2V/dS^2 = N'(d1)/{S*(sigma)*(sqrt(dt))}
5. Vega = d^2V/d sigma^2 = S N'(d1) sqrt(dt)
6. Roh(Call) = dC/dr
7. Roh(Put) = dP/dr

In [ ]:
greek_definition = (
    r"\textbf{Option Greeks} \\[1ex] "
    r"\text{Option Greeks measure an option's price sensitivity to various factors (Black-Scholes).}"
)
greek_formulas = (
    r"\textbf{Option Greeks Formulas} \\[1ex] "
    r"\text{Call: } \Delta_C = N(d_1), \quad "
    r"\Gamma_C = \frac{N'(d_1)}{S \sigma \sqrt{T}}, \quad "
    r"\mathcal{V}_C = S N'(d_1) \sqrt{T}, \quad "
    r"\Theta_C = -\frac{S N'(d_1) \sigma}{2 \sqrt{T}} - r K e^{-r T} N(d_2), \quad "
    r"\rho_C = K T e^{-r T} N(d_2) \\[1ex] "
    r"\text{Put: } \Delta_P = N(d_1) - 1, \quad "
    r"\Gamma_P = \frac{N'(d_1)}{S \sigma \sqrt{T}}, \quad "
    r"\mathcal{V}_P = S N'(d_1) \sqrt{T}, \quad "
    r"\Theta_P = -\frac{S N'(d_1) \sigma}{2 \sqrt{T}} + r K e^{-r T} N(-d_2), \quad "
    r"\rho_P = -K T e^{-r T} N(-d_2) \\[1ex] "
    r"\text{where } d_1 = \frac{\ln(S/K) + (r + \frac{\sigma^2}{2})T}{\sigma \sqrt{T}}, \quad d_2 = d_1 - \sigma \sqrt{T}"
)

render_latex(greek_definition)
render_latex(greek_formulas)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Derivation of Heston PDE using replicating portfolio:

In [ ]:

render_latex(r"""
\textbf{Heston PDE Derivation with Replicating Portfolio} \\[20pt]

\text{Assets:} \\[10pt]
1. \, S_t: \text{Stock price}, \\
\quad dS_t = r S_t dt + \sqrt{v_t} S_t dW_t^{S,\mathbb{Q}} \\[15pt]
2. \, U(S_t, v_t, t): \text{Second option price} \\[15pt]
3. \, B_t = e^{rt}: \text{Risk-free asset}, \\
\quad dB_t = r B_t dt \\[20pt]

\text{Variance dynamics:} \\
\quad dv_t = \kappa (\theta - v_t) dt + \sigma \sqrt{v_t} dW_t^{v,\mathbb{Q}}, \\
\quad dW_t^{S,\mathbb{Q}} dW_t^{v,\mathbb{Q}} = \rho dt \\[20pt]

\text{Option to price:} \\
\quad V(S_t, v_t, t) \\[20pt]

\text{Portfolio:} \\
\quad \Pi_t = \Delta_t S_t + \Gamma_t U_t + \Psi_t B_t = V(S_t, v_t, t) \\[20pt]

\text{Apply Itô’s lemma to } V: \\
\quad dV = \left( \frac{\partial V}{\partial t} + r S_t \frac{\partial V}{\partial S} + \kappa (\theta - v_t) \frac{\partial V}{\partial v} + \frac{1}{2} v_t S_t^2 \frac{\partial^2 V}{\partial S^2} + \frac{1}{2} \sigma^2 v_t \frac{\partial^2 V}{\partial v^2} + \rho \sigma v_t S_t \frac{\partial^2 V}{\partial S \partial v} \right) dt \\
\quad + \sqrt{v_t} S_t \frac{\partial V}{\partial S} dW_t^{S,\mathbb{Q}} + \sigma \sqrt{v_t} \frac{\partial V}{\partial v} dW_t^{v,\mathbb{Q}} \\[20pt]

\text{Similarly for } U: \\
\quad dU = \left( \frac{\partial U}{\partial t} + r S_t \frac{\partial U}{\partial S} + \kappa (\theta - v_t) \frac{\partial U}{\partial v} + \frac{1}{2} v_t S_t^2 \frac{\partial^2 U}{\partial S^2} + \frac{1}{2} \sigma^2 v_t \frac{\partial^2 U}{\partial v^2} + \rho \sigma v_t S_t \frac{\partial^2 U}{\partial S \partial v} \right) dt \\
\quad + \sqrt{v_t} S_t \frac{\partial U}{\partial S} dW_t^{S,\mathbb{Q}} + \sigma \sqrt{v_t} \frac{\partial U}{\partial v} dW_t^{v,\mathbb{Q}} \\[20pt]

\text{Portfolio differential:} \\
\quad d\Pi_t = \Delta_t dS_t + \Gamma_t dU_t + \Psi_t r B_t dt \\[20pt]

\text{Risk-neutral condition:} \\
\quad d\Pi_t = r \Pi_t dt \\[20pt]

\text{Hedge stochastic terms:} \\
\quad 1. \, \Delta_t + \Gamma_t \frac{\partial U}{\partial S} = \frac{\partial V}{\partial S} \\[10pt]
\quad 2. \, \Gamma_t \frac{\partial U}{\partial v} = \frac{\partial V}{\partial v} \\[20pt]

\text{Solve:} \\
\quad \Gamma_t = \frac{\frac{\partial V}{\partial v}}{\frac{\partial U}{\partial v}}, \quad \Delta_t = \frac{\partial V}{\partial S} - \Gamma_t \frac{\partial U}{\partial S} \\[20pt]

\text{Equate drifts and simplify:} \\
\quad \frac{\partial V}{\partial t} + r S_t \frac{\partial V}{\partial S} + \kappa (\theta - v_t) \frac{\partial V}{\partial v} + \frac{1}{2} v_t S_t^2 \frac{\partial^2 V}{\partial S^2} + \frac{1}{2} \sigma^2 v_t \frac{\partial^2 V}{\partial v^2} + \rho \sigma v_t S_t \frac{\partial^2 V}{\partial S \partial v} - r V = 0
""")

<IPython.core.display.Math object>

FFT computes two sums simultanously, resulting in O(nlong(n)) time complexity.

Fourier Pricing is an approximation as the closed form analytical solutions for Heston and Merton are unattainable.

In [ ]:
cf_expression = (
    r"\textbf{Characteristic Function (CF)} \\[1ex] "
    r"\text{Let random variable } X \text{ have Probability Density Function } PDF(x). \text{ Then, the characteristic function (CF), } \hat{p}(u), \text{ is:} \\[1ex] "
    r"\hat{p}(u) = \int_{-\infty}^{\infty} e^{i u x} p(x) \, dx = \mathbb{E}\left[e^{i u X}\right]"
)

render_latex(cf_expression)

<IPython.core.display.Math object>

In [ ]:
cf_definition = (
    r"\begin{aligned} "
    r"&\textbf{Characteristic Function (CF)} \\[2ex] "
    r"&\text{A characteristic function (CF) is a fundamental concept in probability theory and statistics} \\ "
    r"&\text{that describes the distribution of a random variable } X \text{ in the frequency domain.} \\[2ex] "
    r"&\text{It is defined as the expected value of the complex exponential } e^{i u X}, \\ "
    r"&\text{where } u \text{ is a real number (often a frequency or transform variable), and } i = \sqrt{-1} \\ "
    r"&\text{is the imaginary unit.} \\[2ex] "
    r"&\text{Mathematically:} \\[1ex] "
    r"&\hat{p}(u) = \mathbb{E}\left[e^{i u X}\right] = \int_{-\infty}^{\infty} e^{i u x} p(x) \, dx "
    r"\end{aligned}"
)

cf_components = (
    r"\begin{aligned} "
    r"&\textbf{Components:} \\[1ex] "
    r"&\begin{array}{ll} "
    r"p(x) & \text{: The probability density function (PDF) of } X \\[1ex] "
    r"e^{i u x} & \text{: A complex exponential that oscillates with frequency } u \\ "
    r"& \quad \text{as } x \text{ varies} \\[1ex] "
    r"\mathbb{E}[\cdot] & \text{: The expectation operator, which for a continuous random variable} \\ "
    r"& \quad \text{is the integral over the PDF} "
    r"\end{array} "
    r"\end{aligned}"
)

cf_properties = (
    r"\begin{aligned} "
    r"&\textbf{Key Properties of CFs} \\[2ex] "
    r"&1. \textbf{Uniqueness:} \text{ The CF uniquely determines the distribution of } X. \\ "
    r"&\quad \text{If two random variables have the same CF, they have the same distribution (inversion theorem).} \\[1ex] "
    r"&2. \textbf{Existence:} \text{ The CF always exists because } |e^{iu x}| = 1, \\ "
    r"&\quad \text{making the expectation well-defined for any } u. \\[1ex] "
    r"&3. \textbf{Moments:} \text{ Derivatives of the CF at } u=0 \text{ yield the moments of } X: \\ "
    r"&\quad \hat{p}'(0)=i\mathbb{E}[X], \quad \hat{p}''(0)=-\mathbb{E}[X^2]. \\[1ex] "
    r"&4. \textbf{Convolution:} \text{ The CF of the sum of independent random variables is} \\ "
    r"&\quad \text{the product of their individual CFs, useful in option pricing models like Bates.} "
    r"\end{aligned}"
)

cf_interpretation = (
    r"\begin{aligned} "
    r"&\textbf{Intuitive Interpretation} \\[2ex] "
    r"&\text{Think of the CF as a ``simply fourier transformation'' of the random variable's distribution.} \\[1ex] "
    r"&\text{Instead of working directly with the PDF } p(x), \text{ which might be complicated or unavailable,} \\ "
    r"&\text{the CF transforms it into a function of } u \text{ that encodes all distributional information} \\ "
    r"&\text{in a compact, oscillatory form.} "
    r"\end{aligned}"
)

render_latex(cf_definition)
render_latex(cf_components)
render_latex(cf_properties)
render_latex(cf_interpretation)


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

solving Heston PDE:

In [ ]:

render_latex(r"""
\textbf{Heston Pricing via Lewis (2001)} \\[30pt]

\text{Baseline Pricing Equation (Lewis, 2001):} \\[15pt]
\quad C_0 = S_0 - \frac{\sqrt{S_0 K} e^{-r T}}{\pi} \int_0^\infty \text{Re} \left[ \frac{e^{i z k} \phi(z - i/2)}{z^2 + \frac{1}{4}} \right] dz, \quad k = \log \left( \frac{S_0}{K} \right) \\[30pt]

\text{Components Needed:} \\[15pt]
\quad 1. \, \text{Characteristic function } \phi(z - i/2) \text{ for the Heston model.} \\[10pt]
\quad 2. \, \text{Numerical method to compute the integral (e.g., quadrature).} \\[30pt]

\text{Objective:} \\[15pt]
\quad \text{Derive } \phi_H(u, T) \text{ for the Heston model and apply it in the Lewis formula.} \\[30pt]

\text{Heston Model Dynamics:} \\[15pt]
\quad dS_t = r S_t \, dt + \sqrt{\nu_t} S_t \, dW_t^S \\[10pt]
\quad d\nu_t = \kappa_\nu (\theta_\nu - \nu_t) \, dt + \sigma_\nu \sqrt{\nu_t} \, dW_t^\nu \\[10pt]
\quad dW_t^S dW_t^\nu = \rho \, dt \\[10pt]
\quad \text{Where } \nu_0 \text{ is the initial variance, and } s_T = \log S_T. \\[30pt]

\text{Characteristic Function Derivation:} \\[15pt]
\quad \phi_H(u, T) = \mathbb{E}^\mathbb{Q} [ e^{i u s_T} | s_0, \nu_0 ], \quad s_0 = \log S_0 \\[20pt]
\quad \text{Assume a form:} \\[15pt]
\quad \phi_H(u, T) = e^{H_1(u, T) + H_2(u, T) \nu_0} \\[30pt]

\text{Log Price Dynamics:} \\[15pt]
\quad \text{Apply It\^o’s lemma to } s_t = \log S_t: \\[10pt]
\quad ds_t = \left( r - \frac{1}{2} \nu_t \right) dt + \sqrt{\nu_t} dW_t^S \\[15pt]
\quad s_T = s_0 + \int_0^T \left( r - \frac{1}{2} \nu_t \right) dt + \int_0^T \sqrt{\nu_t} dW_t^S \\[30pt]

\text{Kolmogorov Backward Equation:} \\[15pt]
\quad \frac{\partial \phi}{\partial t} + \left( r - \frac{1}{2} \nu_t \right) \frac{\partial \phi}{\partial s} + \kappa_\nu (\theta_\nu - \nu_t) \frac{\partial \phi}{\partial \nu} + \frac{1}{2} \nu_t \frac{\partial^2 \phi}{\partial s^2} + \frac{1}{2} \sigma_\nu^2 \nu_t \frac{\partial^2 \phi}{\partial \nu^2} + \rho \sigma_\nu \nu_t \frac{\partial^2 \phi}{\partial s \partial \nu} = 0 \\[15pt]
\quad \text{Boundary condition:} \\[10pt]
\quad \phi(u, T, s_T, \nu_T) = e^{i u s_T} \\[30pt]

\text{Substitute Assumed Form:} \\[15pt]
\quad \frac{\partial \phi}{\partial t} = \left( \frac{\partial H_1}{\partial t} + \frac{\partial H_2}{\partial t} \nu_t \right) \phi \\[10pt]
\quad \frac{\partial \phi}{\partial s} = i u \phi \\[10pt]
\quad \frac{\partial \phi}{\partial \nu} = H_2 \phi \\[10pt]
\quad \frac{\partial^2 \phi}{\partial s^2} = -u^2 \phi \\[10pt]
\quad \frac{\partial^2 \phi}{\partial \nu^2} = H_2^2 \phi \\[10pt]
\quad \frac{\partial^2 \phi}{\partial s \partial \nu} = i u H_2 \phi \\[15pt]
\quad \text{Substitute into PDE:} \\[10pt]
\quad \left( \frac{\partial H_1}{\partial t} + \frac{\partial H_2}{\partial t} \nu_t \right) + i u \left( r - \frac{1}{2} \nu_t \right) + \kappa_\nu (\theta_\nu - \nu_t) H_2 + \frac{1}{2} \nu_t (-u^2) + \frac{1}{2} \sigma_\nu^2 \nu_t H_2^2 + \rho \sigma_\nu \nu_t i u H_2 = 0 \\[30pt]

\text{Separate Terms:} \\[15pt]
\quad \frac{\partial H_1}{\partial t} + i u r + \kappa_\nu \theta_\nu H_2 = 0 \\[15pt]
\quad \frac{\partial H_2}{\partial t} - \frac{1}{2} i u \nu_t - \kappa_\nu \nu_t H_2 - \frac{1}{2} u^2 \nu_t + \frac{1}{2} \sigma_\nu^2 \nu_t H_2^2 + \rho \sigma_\nu i u \nu_t H_2 = 0 \\[15pt]
\quad \text{Simplify } H_2 \text{ by factoring out } \nu_t: \\[10pt]
\quad \frac{\partial H_2}{\partial t} + \left( -\frac{1}{2} i u - \kappa_\nu H_2 - \frac{1}{2} u^2 + \frac{1}{2} \sigma_\nu^2 H_2^2 + \rho \sigma_\nu i u H_2 \right) \nu_t = 0 \\[30pt]

\text{Solve ODEs:} \\[15pt]
\quad \text{For } H_1: \\[10pt]
\quad \frac{\partial H_1}{\partial t} = -i u r - c_1 H_2, \quad c_1 = \kappa_\nu \theta_\nu \\[15pt]
\quad H_1(u, T) = i u r T + c_1 \int_0^T H_2(u, t) \, dt \\[30pt]
\quad \text{For } H_2: \\[15pt]
\quad \frac{\partial H_2}{\partial t} = \frac{1}{2} (u^2 + i u) + (\kappa_\nu - \rho \sigma_\nu i u) H_2 - \frac{1}{2} \sigma_\nu^2 H_2^2 \\[15pt]
\quad \text{Define:} \\[10pt]
\quad c_2 = -\sqrt{(\rho \sigma_\nu i u - \kappa_\nu)^2 - \sigma_\nu^2 (-i u - u^2)} \\[10pt]
\quad c_3 = \frac{\kappa_\nu - \rho \sigma_\nu i u + c_2}{\kappa_\nu - \rho \sigma_\nu i u - c_2} \\[15pt]
\quad \text{Solution:} \\[10pt]
\quad H_2(u, T) = \frac{\kappa_\nu - \rho \sigma_\nu i u + c_2}{\sigma_\nu^2} \frac{1 - e^{c_2 T}}{1 - c_3 e^{c_2 T}} \\[15pt]
\quad H_1(u, T) = i u r T + \frac{c_1}{\sigma_\nu^2} \left[ (\kappa_\nu - \rho \sigma_\nu i u + c_2) T - 2 \log \left( \frac{1 - c_3 e^{c_2 T}}{1 - c_3} \right) \right] \\[30pt]

\text{Final Characteristic Function:} \\[15pt]
\quad \phi_H(u, T) = e^{H_1(u, T) + H_2(u, T) \nu_0} \\[30pt]

\text{Apply to Lewis Formula:} \\[15pt]
\quad C_0 = S_0 - \frac{\sqrt{S_0 K} e^{-r T}}{\pi} \int_0^\infty \text{Re} \left[ \frac{e^{i z k} \phi_H(z - i/2, T)}{z^2 + \frac{1}{4}} \right] dz
""")

<IPython.core.display.Math object>

Deriving Merton PDE:

In [ ]:
# Merton PDE Derivation with Replicating Portfolio
render_latex(r"""
\textbf{Merton Jump-Diffusion PDE Derivation with Replicating Portfolio} \\[20pt]

\text{Stock Dynamics (Risk-Neutral):} \\
\quad dS_t = (r - \lambda k) S_t dt + \sigma S_t dW_t^\mathbb{Q} + (J - 1) S_t dN_t \\[15pt]
\text{Where:} \\
\quad r: \text{Risk-free rate}, \quad \sigma: \text{Volatility}, \quad W_t^\mathbb{Q}: \text{Brownian motion}, \\
\quad N_t: \text{Poisson process with intensity } \lambda, \\
\quad J: \text{Jump size (lognormal, } \ln J \sim \mathcal{N}(\mu_J, \sigma_J^2)), \quad k = \mathbb{E}[J - 1] = e^{\mu_J + \frac{1}{2} \sigma_J^2} - 1 \\[20pt]

\text{Assets:} \\
\quad 1. \, S_t: \text{Stock price} \\[15pt]
\quad 2. \, B_t = e^{rt}: \text{Risk-free asset}, \quad dB_t = r B_t dt \\[20pt]

\text{Option Price:} \\
\quad V(S_t, t) \\[20pt]

\text{Portfolio:} \\
\quad \Pi_t = \Delta_t S_t + \Psi_t B_t = V(S_t, t) \\[20pt]

\text{Stock Price Change:} \\
\quad \text{If a jump occurs, } S_t \to J S_t, \text{ so } V(S_t, t) \to V(J S_t, t), \\
\quad \Delta S_t = (J - 1) S_t, \quad \Delta V = V(J S_t, t) - V(S_t, t) \\[20pt]

\text{Apply Itô’s lemma (with jumps):} \\
\quad dV = \left( \frac{\partial V}{\partial t} + (r - \lambda k) S_t \frac{\partial V}{\partial S} + \frac{1}{2} \sigma^2 S_t^2 \frac{\partial^2 V}{\partial S^2} \right) dt + \sigma S_t \frac{\partial V}{\partial S} dW_t^\mathbb{Q} + [V(J S_t, t) - V(S_t, t)] dN_t \\[20pt]

\text{Portfolio Differential:} \\
\quad d\Pi_t = \Delta_t dS_t + \Psi_t r B_t dt \\
\quad = \Delta_t \left[ (r - \lambda k) S_t dt + \sigma S_t dW_t^\mathbb{Q} + (J - 1) S_t dN_t \right] + \Psi_t r B_t dt \\[20pt]

\text{Risk-Neutral Condition:} \\
\quad d\Pi_t = r \Pi_t dt = r (V - \Delta_t S_t) dt \\[20pt]

\text{Hedge Continuous Risk:} \\
\quad \text{Coefficient of } dW_t^\mathbb{Q}: \quad \Delta_t \sigma S_t = \sigma S_t \frac{\partial V}{\partial S} \\
\quad \Delta_t = \frac{\partial V}{\partial S} \\[20pt]

\text{Substitute } \Delta_t: \\
\quad d\Pi_t = \frac{\partial V}{\partial S} \left[ (r - \lambda k) S_t dt + \sigma S_t dW_t^\mathbb{Q} + (J - 1) S_t dN_t \right] + r (V - \frac{\partial V}{\partial S} S_t) dt \\[20pt]

\text{Equate to } r V dt \text{ and simplify (dt terms):} \\
\quad \frac{\partial V}{\partial t} + (r - \lambda k) S_t \frac{\partial V}{\partial S} + \frac{1}{2} \sigma^2 S_t^2 \frac{\partial^2 V}{\partial S^2} + \lambda \mathbb{E} [V(J S_t, t) - V(S_t, t)] - r V = 0 \\[20pt]

\text{Final Merton PDE:} \\
\quad \frac{\partial V}{\partial t} + (r - \lambda k) S_t \frac{\partial V}{\partial S} + \frac{1}{2} \sigma^2 S_t^2 \frac{\partial^2 V}{\partial S^2} + \lambda \int_0^\infty [V(J S_t, t) - V(S_t, t)] f(J) dJ - r V = 0
""")

<IPython.core.display.Math object>

In [ ]:

render_latex(r"""
\textbf{Merton (1976) via Lewis (2001)} \\[30pt]

\text{Lewis (2001) Pricing Formula:} \\[15pt]
\quad C_0 = S_0 - \frac{\sqrt{S_0 K} e^{-r T}}{\pi} \int_0^\infty \text{Re} \left[ \frac{e^{i z k} \phi(z - i/2)}{z^2 + \frac{1}{4}} \right] dz, \quad k = \log \left( \frac{S_0}{K} \right) \\[30pt]

\text{Merton Characteristic Function:} \\[15pt]
\quad \phi_{M76}(u, T) = e^{i u \omega - \frac{u^2 \sigma^2}{2} T + \lambda \left( e^{i u \mu_J - \frac{u^2 \delta^2}{2}} - 1 \right) T} \\[15pt]
\quad \text{Where:} \\[10pt]
\quad \omega = r - \frac{\sigma^2}{2} - \lambda \left( e^{\mu_J + \frac{\delta^2}{2}} - 1 \right) \\[30pt]

\text{Derivation of } \phi_{M76}(u, T): \\[15pt]
\quad s_t = \log S_t, \quad ds_t = \left( r - r_J - \frac{1}{2} \sigma^2 \right) dt + \sigma dZ_t + \log(1 + J_t) dN_t \\[15pt]
\quad s_T = s_0 + \left( r - r_J - \frac{1}{2} \sigma^2 \right) T + \sigma Z_T + \sum_{j=1}^{N_T} \log(1 + J_j) \\[15pt]
\quad \phi(u, T) = \mathbb{E}^\mathbb{Q} [ e^{i u s_T} ] = e^{i u s_0} \mathbb{E}^\mathbb{Q} \left[ e^{i u \left[ (r - r_J - \frac{1}{2} \sigma^2) T + \sigma Z_T + \sum_{j=1}^{N_T} \log(1 + J_j) \right]} \right] \\[30pt]

\text{Expectations:} \\[15pt]
\quad \text{Diffusion term:} \quad \mathbb{E} [ e^{i u \sigma Z_T} ] = e^{-\frac{u^2 \sigma^2}{2} T}, \quad Z_T \sim N(0, T) \\[15pt]
\quad \text{Jump term:} \\[10pt]
\quad \mathbb{E} [ e^{i u \sum_{j=1}^{N_T} \log(1 + J_j)} ] = \sum_{n=0}^\infty \frac{e^{-\lambda T} (\lambda T)^n}{n!} \left( \mathbb{E} [ e^{i u \log(1 + J)} ] \right)^n \\[15pt]
\quad \log(1 + J_t) \sim N \left( \log(1 + \mu_J) - \frac{\delta^2}{2}, \delta^2 \right) \\[10pt]
\quad \mathbb{E} [ e^{i u \log(1 + J)} ] = e^{i u \left( \log(1 + \mu_J) - \frac{\delta^2}{2} \right) - \frac{u^2 \delta^2}{2}} = \frac{e^{i u \mu_J - \frac{u^2 \delta^2}{2}}}{(1 + \mu_J)^{-i u}} \\[15pt]
\quad \text{Adjust for Merton’s form:} \quad e^{i u \mu_J - \frac{u^2 \delta^2}{2}} \\[15pt]
\quad \text{Poisson sum:} \quad \mathbb{E} [ e^{i u \sum \log(1 + J_j)} ] = e^{\lambda T \left( e^{i u \mu_J - \frac{u^2 \delta^2}{2}} - 1 \right)} \\[30pt]

\text{Combine:} \\[15pt]
\quad \phi_{M76}(u, T) = e^{i u \left[ s_0 + (r - r_J - \frac{1}{2} \sigma^2) T \right] - \frac{u^2 \sigma^2}{2} T + \lambda T \left( e^{i u \mu_J - \frac{u^2 \delta^2}{2}} - 1 \right)} \\[15pt]
\quad \text{Substitute } r_J = \lambda \left( e^{\mu_J + \frac{\delta^2}{2}} - 1 \right): \\[10pt]
\quad \omega = r - \frac{\sigma^2}{2} - \lambda \left( e^{\mu_J + \frac{\delta^2}{2}} - 1 \right) \\[15pt]
\quad \phi_{M76}(u, T) = e^{i u \omega T - \frac{u^2 \sigma^2}{2} T + \lambda T \left( e^{i u \mu_J - \frac{u^2 \delta^2}{2}} - 1 \right)} \\[30pt]

\text{Apply to Lewis:} \\[15pt]
\quad \phi(z - i/2, T) = e^{i (z - i/2) \omega T - \frac{(z - i/2)^2 \sigma^2}{2} T + \lambda T \left( e^{i (z - i/2) \mu_J - \frac{(z - i/2)^2 \delta^2}{2}} - 1 \right)} \\[15pt]
\quad C_0 = S_0 - \frac{\sqrt{S_0 K} e^{-r T}}{\pi} \int_0^\infty \text{Re} \left[ \frac{e^{i z k} \phi_{M76}(z - i/2, T)}{z^2 + \frac{1}{4}} \right] dz
""")

<IPython.core.display.Math object>

In [ ]:

render_latex(r"""
\textbf{Merton (1976) via Carr-Madan (1999)} \\[30pt]

\text{Merton Risk-Neutral Dynamics:} \\[15pt]
\quad dS_t = (r - r_J) S_t \, dt + \sigma S_t \, dZ_t + J_t S_t \, dN_t \\[15pt]
\quad \text{Where:} \\[10pt]
\quad r: \text{constant risk-free rate,} \\[10pt]
\quad r_J = \lambda \left( e^{\mu_J + \frac{\delta^2}{2}} - 1 \right): \text{drift correction,} \\[10pt]
\quad \sigma: \text{constant volatility,} \\[10pt]
\quad Z_t: \text{standard Brownian motion,} \\[10pt]
\quad J_t: \text{jump size, } \log(1 + J_t) \sim N \left( \log(1 + \mu_J) - \frac{\delta^2}{2}, \delta^2 \right), \\[10pt]
\quad N_t: \text{Poisson process with intensity } \lambda \\[30pt]

\text{Carr-Madan (1999) Pricing Formula:} \\[15pt]
\quad C_0 = \frac{e^{-\alpha k}}{\pi} \int_0^\infty e^{-i \nu k} \Psi(\nu) \, d\nu \\[15pt]
\quad \text{Where:} \\[10pt]
\quad k = \log K, \\[10pt]
\quad \Psi(\nu) = \frac{e^{-r T} \phi(\nu - (\alpha + 1)i)}{\alpha^2 + \alpha - \nu^2 + i (2\alpha + 1) \nu}, \\[10pt]
\quad \phi(u) = \mathbb{E}^\mathbb{Q} [ e^{i u s_T} ], \quad s_T = \log S_T \\[30pt]

\text{Merton Characteristic Function:} \\[15pt]
\quad \phi_{M76}(u, T) = e^{i u \omega T - \frac{u^2 \sigma^2}{2} T + \lambda T \left( e^{i u \mu_J - \frac{u^2 \delta^2}{2}} - 1 \right)} \\[15pt]
\quad \text{Where:} \\[10pt]
\quad \omega = r - \frac{\sigma^2}{2} - \lambda \left( e^{\mu_J + \frac{\delta^2}{2}} - 1 \right) \\[30pt]

\text{Derivation of } \phi_{M76}(u, T): \\[15pt]
\quad s_t = \log S_t, \\[10pt]
\quad ds_t = \left( r - r_J - \frac{1}{2} \sigma^2 \right) dt + \sigma dZ_t + \log(1 + J_t) dN_t \\[15pt]
\quad s_T = s_0 + \left( r - r_J - \frac{1}{2} \sigma^2 \right) T + \sigma Z_T + \sum_{j=1}^{N_T} \log(1 + J_j) \\[15pt]
\quad \phi(u, T) = e^{i u s_0} \mathbb{E}^\mathbb{Q} \left[ e^{i u \left[ (r - r_J - \frac{1}{2} \sigma^2) T + \sigma Z_T + \sum_{j=1}^{N_T} \log(1 + J_j) \right]} \right] \\[15pt]
\quad \text{Diffusion:} \quad \mathbb{E} [ e^{i u \sigma Z_T} ] = e^{-\frac{u^2 \sigma^2}{2} T} \\[10pt]
\quad \text{Jumps:} \quad \mathbb{E} [ e^{i u \sum \log(1 + J_j)} ] = e^{\lambda T \left( e^{i u \mu_J - \frac{u^2 \delta^2}{2}} - 1 \right)} \\[15pt]
\quad \phi_{M76}(u, T) = e^{i u (s_0 + \omega T) - \frac{u^2 \sigma^2}{2} T + \lambda T \left( e^{i u \mu_J - \frac{u^2 \delta^2}{2}} - 1 \right)} \\[30pt]

\text{Apply to Carr-Madan:} \\[15pt]
\quad \text{Compute } \phi(\nu - (\alpha + 1)i): \\[10pt]
\quad \phi(\nu - (\alpha + 1)i, T) = e^{i (\nu - (\alpha + 1)i) \omega T - \frac{(\nu - (\alpha + 1)i)^2 \sigma^2}{2} T + \lambda T \left( e^{i (\nu - (\alpha + 1)i) \mu_J - \frac{(\nu - (\alpha + 1)i)^2 \delta^2}{2}} - 1 \right)} \\[15pt]
\quad \Psi(\nu) = \frac{e^{-r T} \phi(\nu - (\alpha + 1)i)}{\alpha^2 + \alpha - \nu^2 + i (2\alpha + 1) \nu} \\[15pt]
\quad C_0 = \frac{e^{-\alpha k}}{\pi} \int_0^\infty e^{-i \nu k} \Psi(\nu) \, d\nu \\[30pt]

\text{Additional Note:} \\[15pt]
\quad \text{The Carr-Madan method damps the option price with } e^{\alpha k} \text{ to ensure integrability, then uses the Fourier transform of this damped price, inverted via } \Psi(\nu).
""")

<IPython.core.display.Math object>

In [ ]:

carr_madan_damping_process = (
    r"\text{Carr-Madan: Damping with } e^{\alpha k} \text{ (} k = \ln(K/S_0), \alpha > 0 \text{) makes } C(k) \text{ integrable. Fourier transform of } e^{\alpha k} C(k) \text{ gives } \Psi(\nu), \text{ inverted as } C(k) = e^{-\alpha k} \times \text{FFT of } \Psi(\nu) \text{ using the log-price CF.}"
)

render_latex(carr_madan_damping_process)

<IPython.core.display.Math object>

In [ ]:
Number = Union[float, int]

def Heston_characteristic_function(u: Number, T: float, r: float, kappa_v: float, theta_v: float,
                    sigma_v: float, rho: float, v0: float) -> complex:
    """
    Valuation of European call option in Heston model via Lewis (2001)
    Fourier-based approach: characteristic function.

    Parameter definitions see function heston_call.
    """
    c1: float = kappa_v * theta_v
    c2: complex = -np.sqrt(
        (rho * sigma_v * u * 1j - kappa_v) ** 2 - sigma_v**2 * (-u * 1j - u**2)
    )
    c3: complex = (kappa_v - rho * sigma_v * u * 1j + c2) / (
        kappa_v - rho * sigma_v * u * 1j - c2
    )
    H1: complex = r * u * 1j * T + (c1 / sigma_v**2) * (
        (kappa_v - rho * sigma_v * u * 1j + c2) * T
        - 2 * np.log((1 - c3 * np.exp(c2 * T)) / (1 - c3))
    )
    H2: complex = ((kappa_v - rho * sigma_v * u * 1j + c2) / sigma_v**2) * (
        (1 - np.exp(c2 * T)) / (1 - c3 * np.exp(c2 * T))
    )
    char_func_value: complex = np.exp(H1 + H2 * v0)
    return char_func_value

def heston_integration_function(u: float, S0: float, K: float, T: float, r: float,
                   kappa_v: float, theta_v: float, sigma_v: float,
                   rho: float, v0: float) -> float:
    """
    Fourier-based approach for Lewis (2001): Integration function.
    """
    # Shiftting u to obtain the appropriate integration variable char func.
    char_func_value: complex = Heston_characteristic_function(
        u - 1j * 0.5, T, r, kappa_v, theta_v, sigma_v, rho, v0
    )
    integrand: complex = np.exp(1j * u * np.log(S0 / K)) * char_func_value
    int_func_value: float = (1 / (u**2 + 0.25)) * integrand.real
    return int_func_value

def heston_call(S0: float, K: float, T: float, r: float,
                   kappa_v: float, theta_v: float, sigma_v: float,
                   rho: float, v0: float) -> float:
    """
    Valuation of European call option in H93 model via Lewis (2001)

    Parameter definition:
    ==========
    S0: float
        initial stock/index level
    K: float
        strike price
    T: float
        time-to-maturity (for t=0)
    r: float
        constant risk-free short rate
    kappa_v: float
        mean-reversion factor
    theta_v: float
        long-run mean of variance
    sigma_v: float
        volatility of variance
    rho: float
        correlation between variance and stock/index level
    v0: float
        initial level of variance

    Returns
    =======
    call_value: float
        present value of European call option
    """
    int_value: float = quad(
        lambda u: heston_integration_function(u, S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0),
        0,
        np.inf,
        limit=250,
    )[0]
    call_value: float = max(0, S0 - np.exp(-r * T) * np.sqrt(S0 * K) / np.pi * int_value)
    return call_value


In [ ]:
# Option Parameters
S0 = 80.0       # Initial stock price
K = 110.0       # Strike price of the option
T = 0.5         # Time to expiration in years
r = 0.02        # Risk-free interest rate

# Heston(1993) Parameters
kappa_v = 1.5   # Speed of mean reversion for volatility
theta_v = 0.02  # Long-term average volatility
sigma_v = 0.75  # Volatility of volatility (how much volatility fluctuates)
rho = 0.1       # Correlation between stock price and volatility
v0 = 0.01       # Initial variance (starting volatility squared)

In [ ]:
print(f"Heston (1993) Call Option Value: {heston_call(S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0)}")

Heston (1993) Call Option Value: 0.0663405482251136


In [ ]:
def merton_characteristic_function(u: Number, T: float, r: float, sigma: float,
                                     lamb: float, mu: float, delta: float) -> complex:
    """
    Characteristic function for Merton '76 model.

    Parameters:
    -----------
    u : Number
        Complex or real argument.
    T : float
        Time-to-maturity.
    r : float
        Risk-free interest rate.
    sigma : float
        Diffusion volatility.
    lamb : float
        Jump intensity.
    mu : float
        Mean of jump size.
    delta : float
        Standard deviation of jump size.

    Returns:
    --------
    complex
        The value of the characteristic function.
    """
    omega: float = r - 0.5 * sigma**2 - lamb * (np.exp(mu + 0.5 * delta**2) - 1)
    char_func_value: complex = np.exp(
        (
            1j * u * omega
            - 0.5 * u**2 * sigma**2
            + lamb * (np.exp(1j * u * mu - 0.5 * u**2 * delta**2) - 1)
        )
        * T
    )
    return char_func_value

def merton_integration_function(u: float, S0: float, K: float, T: float, r: float,
                                 sigma: float, lamb: float, mu: float, delta: float) -> float:
    """
    Integral function for Lewis (2001) under the Merton '76 characteristic function.

    Parameters:
    -----------
    u : float
        Integration variable.
    S0 : float
        Initial stock price.
    K : float
        Strike price.
    T : float
        Time-to-maturity.
    r : float
        Risk-free interest rate.
    sigma : float
        Diffusion volatility.
    lamb : float
        Jump intensity.
    mu : float
        Mean of jump size.
    delta : float
        Standard deviation of jump size.

    Returns:
    --------
    float
        The value of the integrand.
    """
    # Adjust the integration var for the char function.
    char_func: complex = merton_characteristic_function(u - 0.5 * 1j, T, r, sigma, lamb, mu, delta)
    integrand: complex = np.exp(1j * u * np.log(S0 / K)) * char_func
    value: float = (1 / (u**2 + 0.25)) * integrand.real
    return value

def merton_call_value(S0: float, K: float, T: float, r: float, sigma: float,
                      lamb: float, mu: float, delta: float) -> float:
    """
    Value of a European call option under the Merton '76 jump diffusion model
    using Lewis (2001) Fourier-based approach.

    Parameters:
    -----------
    S0 : float
        Initial stock price.
    K : float
        Strike price.
    T : float
        Time-to-maturity.
    r : float
        Risk-free interest rate.
    sigma : float
        Diffusion volatility.
    lamb : float
        Jump intensity.
    mu : float
        Mean of jump size.
    delta : float
        Standard deviation of jump size.

    Returns:
    --------
    float
        Present value of the European call option.
    """
    int_value: float = quad(
        lambda u: merton_integration_function(u, S0, K, T, r, sigma, lamb, mu, delta),
        0,
        50,
        limit=250,
    )[0]

    call_value: float = max(0, S0 - np.exp(-r * T) * np.sqrt(S0 * K) / np.pi * int_value)
    return call_value

In [ ]:
# Option Parameters
S0 = 100        # Initial stock price
K = 100         # Strike price of the option
T = 1           # Time to expiration in years
r = 0.05        # Risk-free interest rate

# Merton Parameters
sigma = 0.4     # Volatility of the diffusion part (continuous price movement)
lamb = 1        # Jump intensity (average number of jumps per year)
mu = -0.2       # Average jump size (log-return of jumps)
delta = 0.1     # Standard deviation of jump size

In [ ]:
print(f"Value of the Call option under Merton (1976): {merton_call_value(S0, K, T, r, sigma, lamb, mu, delta)}")

Value of the Call option under Merton (1976): 19.947853881446505


In [ ]:
import numpy as np
import pandas as pd
from typing import Callable
from numpy.typing import NDArray

def calibration_error_function(
    p0: NDArray[np.float_],
    S0: float,
    options: pd.DataFrame,
    call_value_func: Callable[[float, float, float, float, ...], float]
) -> float:
    """
    Generic calibration error function for option pricing models (e.g., Heston and Merton).

    Parameters
    ----------
    p0 : NDArray[np.float_]
        Model parameter vector. Its elements should match the parameters expected by call_value_func.
    S0 : float
        Current underlying asset price.
    options : pd.DataFrame
        Market option data with columns:
            'Strike': Strike price,
            'T': Time-to-maturity,
            'r': Risk-free rate,
            'Call': Observed market call option price.
    call_value_func : Callable
        A function that computes the model call option price. Its signature must be:
            call_value_func(S0: float, K: float, T: float, r: float, *params) -> float

    Returns
    -------
    float
        The root mean squared error (RMSE) between the model prices and market prices.
    """
    squared_errors = []
    for _, row in options.iterrows():
        model_price = call_value_func(S0, row["Strike"], row["T"], row["r"], *p0)
        squared_errors.append((model_price - row["Call"]) ** 2)
    rmse = np.sqrt(np.mean(squared_errors))
    return rmse


In [ ]:
def merton_calibration_full(S0: float, options: pd.DataFrame) -> NDArray[np.float_]:
    """
    Calibrates the Merton (1976) jump diffusion model to market quotes using a two-step
    procedure: a global brute-force search followed by local minimization.

    Parameters
    ----------
    S0 : float
        The current underlying asset price.
    options : pd.DataFrame
        Market option data. Expected columns are:
            'Strike': strike price,
            'T': time-to-maturity,
            'r': risk-free interest rate,
            'Call': observed market call option price.

    Returns
    -------
    NDArray[np.float_]
        Optimized parameter vector [sigma, lamb, mu, delta].
    """
    # Defining error function wrapper for Merton model.
    def error_function_merton(p: NDArray[np.float_]) -> float:
        return calibration_error_function(p, S0, options, merton_call_value)

    # Global search with brute force. The slices define the search ranges and step sizes:
    ranges = (
        slice(0.075, 0.201, 0.025),   # sigma: from 0.075 to 0.201 in steps of 0.025
        slice(0.10, 0.401, 0.1),      # lamb: from 0.10 to 0.401 in steps of 0.1
        slice(-0.5, 0.01, 0.1),       # mu: from -0.5 to 0.01 in steps of 0.1
        slice(0.10, 0.301, 0.1),      # delta: from 0.10 to 0.301 in steps of 0.1
    )
    p0: NDArray[np.float_] = brute(error_function_merton, ranges, finish=None)

    # Local search using fmin to refine the solution.
    opt: NDArray[np.float_] = fmin(
        error_function_merton, p0, xtol=0.0001, ftol=0.0001, maxiter=550, maxfun=1050, disp=True
    )
    return opt

In [ ]:
# # Option Parameters
# S0 = 100.0
# K = 100.0
# T = 1.0
# r = 0.02

# # Heston(1993) Parameters
# kappa_v = 1.5
# theta_v = 0.02
# sigma_v = 0.15
# rho = 0.1
# v0 = 0.01

# # Calculate the Heston call price
# heston_price = OptionPricingLibrary.heston_call_value(S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0)
# print("Heston Call Price:", heston_price)


In [ ]:
Number = Union[float, int, complex]

class OptionPricingLibraryHestonMetron:
    _r: float
    _S0: float
    _K: float
    _T: float

    def __new__(self, r: float, S0: float, K: float, T: float) -> Self:
        self = super().__new__(self)
        self._r = r
        self._S0 = S0
        self._K = K
        self._T = T
        return self

    def heston_characteristic_function(self, u: Number, kappa_v: float,
                                         theta_v: float, sigma_v: float, rho: float, v0: float) -> complex:
        """
        Heston characteristic function (Fourier domain) for the European call option.
        """
        c1: float = kappa_v * theta_v
        c2: complex = -np.sqrt((rho * sigma_v * u * 1j - kappa_v) ** 2 - sigma_v**2 * (-u * 1j - u**2))
        c3: complex = (kappa_v - rho * sigma_v * u * 1j + c2) / (kappa_v - rho * sigma_v * u * 1j - c2)
        H1: complex = self._r * u * 1j * self._T + (c1 / sigma_v**2) * (
            (kappa_v - rho * sigma_v * u * 1j + c2) * self._T
            - 2 * np.log((1 - c3 * np.exp(c2 * self._T)) / (1 - c3))
        )
        H2: complex = ((kappa_v - rho * sigma_v * u * 1j + c2) / sigma_v**2) * (
            (1 - np.exp(c2 * self._T)) / (1 - c3 * np.exp(c2 * self._T))
        )
        return np.exp(H1 + H2 * v0)

    def heston_integration_function(self, u: float,
                                     kappa_v: float, theta_v: float, sigma_v: float, rho: float, v0: float) -> float:
        """
        Integration function for the Heston model (Lewis 2001 approach).
        """
        # Note: Adjust the shift by 0.5*1j in the argument
        char_func: complex = self.heston_characteristic_function(u - 0.5 * 1j, kappa_v, theta_v, sigma_v, rho, v0)
        integrand: complex = np.exp(1j * u * np.log(self._S0 / self._K)) * char_func
        return (1 / (u**2 + 0.25)) * integrand.real

    def heston_call_value(self, kappa_v: float,
                          theta_v: float, sigma_v: float, rho: float, v0: float) -> float:
        """
        Price of a European call option using the Heston model.
        """
        int_value: float = quad(
            lambda u: self.heston_integration_function(u, kappa_v, theta_v, sigma_v, rho, v0),
            0,
            np.inf,
            limit=250,
        )[0]
        call_value: float = max(0, self._S0 - np.exp(-self._r * self._T) * np.sqrt(self._S0 * self._K) / np.pi * int_value)
        return call_value

    def merton_characteristic_function(self, u: Number, sigma: float,
                                         lamb: float, mu: float, delta: float) -> complex:
        """
        Merton (1976) characteristic function for the European call option.
        """
        omega: float = self._r - 0.5 * sigma**2 - lamb * (np.exp(mu + 0.5 * delta**2) - 1)
        return np.exp(
            (
                1j * u * omega
                - 0.5 * u**2 * sigma**2
                + lamb * (np.exp(1j * u * mu - 0.5 * u**2 * delta**2) - 1)
            )
            * self._T
        )

    def merton_integration_function(self, u: float,
                                     sigma: float, lamb: float, mu: float, delta: float) -> float:
        """
        Integration function for the Merton model (Lewis 2001 approach).
        """
        char_func: complex = self.merton_characteristic_function(u - 0.5 * 1j, sigma, lamb, mu, delta)
        integrand: complex = np.exp(1j * u * np.log(self._S0 / self._K)) * char_func
        return (1 / (u**2 + 0.25)) * integrand.real

    def merton_call_value(self, sigma: float,
                          lamb: float, mu: float, delta: float) -> float:
        """
        Price of a European call option using the Merton jump diffusion model.
        """
        int_value: float = quad(
            lambda u: self.merton_integration_function(u, sigma, lamb, mu, delta),
            0,
            50,
            limit=250,
        )[0]
        call_value: float = max(0, self._S0 - np.exp(-self._r * self._T) * np.sqrt(self._S0 * self._K) / np.pi * int_value)
        return call_value

    def calibration_error_function(self, p0: NDArray[np.float_], options: pd.DataFrame,
                                   call_value_func: Callable[..., float]) -> float:
        """
        Generic calibration error function that computes the RMSE between model and market call prices.

        Parameters:
        -----------
        p0 : NDArray[np.float_]
            The parameter vector for the model.
        options : pd.DataFrame
            A DataFrame with market option data. Expected columns:
            'Strike', 'T', 'r', and 'Call'.
        call_value_func : Callable[..., float]
            The option pricing function (e.g., merton_call_value or heston_call_value).

        Returns:
        --------
        float
            The RMSE of the model prices relative to market prices.
        """
        squared_errors = []
        for _, row in options.iterrows():

            model_price: float = call_value_func(
                sigma=p0[0],
                lamb=p0[1],
                mu=p0[2],
                delta=p0[3]
            ) if call_value_func.__name__ == 'merton_call_value' else call_value_func(
                kappa_v=p0[0],
                theta_v=p0[1],
                sigma_v=p0[2],
                rho=p0[3],
                v0=p0[4]
            )
            squared_errors.append((model_price - row["Call"]) ** 2)
        rmse: float = np.sqrt(np.mean(squared_errors))
        return rmse

    def calibrate_merton(self, options: pd.DataFrame) -> NDArray[np.float_]:
        """
        Calibrates the Merton (1976) jump diffusion model to market quotes using a two-step
        procedure: a global grid search followed by local minimization.

        Parameters:
        -----------
        options : pd.DataFrame
            Market option data. Expected columns: 'Strike', 'T', 'r', and 'Call'.

        Returns:
        --------
        NDArray[np.float_]
            The optimized parameter vector [sigma, lamb, mu, delta].
        """
        def error_function(p: NDArray[np.float_]) -> float:
            return self.calibration_error_function(p, options, self.merton_call_value)

        # Global search over specified ranges for [sigma, lamb, mu, delta]
        ranges = (
            slice(0.075, 0.201, 0.025),  # sigma
            slice(0.10, 0.401, 0.1),      # lamb
            slice(-0.5, 0.01, 0.1),       # mu
            slice(0.10, 0.301, 0.1),      # delta
        )
        p0: NDArray[np.float_] = brute(error_function, ranges, finish=None)
        opt: NDArray[np.float_] = fmin(
            error_function, p0, xtol=0.0001, ftol=0.0001, maxiter=550, maxfun=1050, disp=True
        )
        return opt



In [ ]:
# Option Parameters
S0 = 100.0      # Initial stock price
K = 100.0       # Strike price of the option
T = 1.0         # Time to expiration in years
r = 0.02        # Risk-free interest rate

# Heston(1993) Parameters
kappa_v = 1.5   # Speed of mean reversion for volatility
theta_v = 0.02  # Long-term average volatility
sigma_v = 0.15  # Volatility of volatility (how much volatility fluctuates)
rho = 0.1       # Correlation between stock price and volatility
v0 = 0.01       # Initial variance (starting volatility squared)


pricing_lib = OptionPricingLibraryHestonMetron(r, S0, K, T)

heston_price = pricing_lib.heston_call_value(kappa_v, theta_v, sigma_v, rho, v0)
print("Heston Call Price:", heston_price)


Heston Call Price: 5.757807070918929


In [ ]:
# Option Parameters
S0 = 100        # Initial stock price
K = 100         # Strike price of the option
T = 1           # Time to expiration in years
r = 0.05        # Risk-free interest rate

# Merton Parameters
sigma = 0.4     # Volatility of the diffusion part (continuous price movement)
lamb = 1        # Jump intensity (average number of jumps per year)
mu = -0.2       # Average jump size (log-return of jumps)
delta = 0.1     # Standard deviation of jump size

pricing_lib = OptionPricingLibraryHestonMetron(r, S0, K, T)

merton_price = pricing_lib.merton_call_value(sigma, lamb, mu, delta)
print("Merton Call Price:", merton_price)


Merton Call Price: 19.947853881446505


Bates:

In [ ]:
# Bates (1996) via Lewis (2001)
render_latex(r"""
\textbf{Bates (1996) via Lewis (2001)} \\[30pt]

\text{Risk-Neutral Dynamics} \\[20pt]
dS_t = (r_t - r_J) S_t \, dt + \sqrt{\nu_t} S_t \, dZ_t^1 + J_t S_t \, dN_t \\[15pt]
d\nu_t = \kappa_\nu (\theta_\nu - \nu_t) \, dt + \sigma_\nu \sqrt{\nu_t} \, dZ_t^2 \\[20pt]

\text{Lewis (2001) Pricing Formula} \\[20pt]
C_0 = S_0 - \frac{\sqrt{S_0 K} e^{-r T}}{\pi} \int_0^\infty \text{Re} \left[ \frac{e^{i z k} \phi_{B96}(z - i/2, T)}{z^2 + \frac{1}{4}} \right] dz \\[15pt]
k = \log \left( \frac{S_0}{K} \right) \\[20pt]

\text{Bates Characteristic Function} \\[20pt]
\phi_{B96}(u, T) = \phi_{H93}(u, T) \cdot \phi_{M76J}(u, T) \\[20pt]
\phi_{H93}(u, T) = e^{H_1(u, T) + H_2(u, T) \nu_0} \\[15pt]
H_1(u, T) = r_0 u i T + \frac{c_1}{\sigma_\nu^2} \left[ (\kappa_\nu - \rho \sigma_\nu u i + c_2) T - 2 \log \left( \frac{1 - c_3 e^{c_2 T}}{1 - c_3} \right) \right] \\[15pt]
H_2(u, T) = \frac{\kappa_\nu - \rho \sigma_\nu u i + c_2}{\sigma_\nu^2} \frac{1 - e^{c_2 T}}{1 - c_3 e^{c_2 T}} \\[15pt]
c_1 = \kappa_\nu \theta_\nu \\[10pt]
c_2 = -\sqrt{(\rho \sigma_\nu u i - \kappa_\nu)^2 - \sigma_\nu^2 (-u i - u^2)} \\[10pt]
c_3 = \frac{\kappa_\nu - \rho \sigma_\nu u i + c_2}{\kappa_\nu - \rho \sigma_\nu u i - c_2} \\[20pt]
\phi_{M76J}(u, T) = e^{i u \omega T + \lambda T \left( e^{i u \mu_J - \frac{u^2 \delta^2}{2}} - 1 \right)} \\[15pt]
\omega = -\lambda \left( e^{\mu_J + \frac{\delta^2}{2}} - 1 \right) \\[20pt]

\text{Final Expression} \\[20pt]
\phi_{B96}(z - i/2, T) = \phi_{H93}(z - i/2, T) \cdot \phi_{M76J}(z - i/2, T) \\[15pt]
C_0 = S_0 - \frac{\sqrt{S_0 K} e^{-r T}}{\pi} \int_0^\infty \text{Re} \left[ \frac{e^{i z k} \phi_{H93}(z - i/2, T) \phi_{M76J}(z - i/2, T)}{z^2 + \frac{1}{4}} \right] dz
""")

<IPython.core.display.Math object>

In [ ]:
# Bates (1996) via Carr-Madan (1999)
render_latex(r"""
\textbf{Bates (1996) via Carr-Madan (1999)} \\[30pt]

\text{Risk-Neutral Dynamics} \\[20pt]
dS_t = (r_t - r_J) S_t \, dt + \sqrt{\nu_t} S_t \, dZ_t^1 + J_t S_t \, dN_t \\[15pt]
d\nu_t = \kappa_\nu (\theta_\nu - \nu_t) \, dt + \sigma_\nu \sqrt{\nu_t} \, dZ_t^2 \\[20pt]

\text{Carr-Madan (1999) Pricing Formula} \\[20pt]
C_0 = \frac{e^{-\alpha k}}{\pi} \int_0^\infty e^{-i \nu k} \frac{e^{-r T} \phi_{B96}(\nu - (\alpha + 1)i, T)}{\alpha^2 + \alpha - \nu^2 + i (2\alpha + 1) \nu} \, d\nu \\[15pt]
k = \log K \\[20pt]

\text{Bates Characteristic Function} \\[20pt]
\phi_{B96}(u, T) = \phi_{H93}(u, T) \cdot \phi_{M76J}(u, T) \\[20pt]
\phi_{H93}(u, T) = e^{H_1(u, T) + H_2(u, T) \nu_0} \\[15pt]
H_1(u, T) = r_0 u i T + \frac{c_1}{\sigma_\nu^2} \left[ (\kappa_\nu - \rho \sigma_\nu u i + c_2) T - 2 \log \left( \frac{1 - c_3 e^{c_2 T}}{1 - c_3} \right) \right] \\[15pt]
H_2(u, T) = \frac{\kappa_\nu - \rho \sigma_\nu u i + c_2}{\sigma_\nu^2} \frac{1 - e^{c_2 T}}{1 - c_3 e^{c_2 T}} \\[15pt]
c_1 = \kappa_\nu \theta_\nu \\[10pt]
c_2 = -\sqrt{(\rho \sigma_\nu u i - \kappa_\nu)^2 - \sigma_\nu^2 (-u i - u^2)} \\[10pt]
c_3 = \frac{\kappa_\nu - \rho \sigma_\nu u i + c_2}{\kappa_\nu - \rho \sigma_\nu u i - c_2} \\[20pt]
\phi_{M76J}(u, T) = e^{i u \omega T + \lambda T \left( e^{i u \mu_J - \frac{u^2 \delta^2}{2}} - 1 \right)} \\[15pt]
\omega = -\lambda \left( e^{\mu_J + \frac{\delta^2}{2}} - 1 \right) \\[20pt]

\text{Final Expression} \\[20pt]
\phi_{B96}(\nu - (\alpha + 1)i, T) = \phi_{H93}(\nu - (\alpha + 1)i, T) \cdot \phi_{M76J}(\nu - (\alpha + 1)i, T) \\[15pt]
C_0 = \frac{e^{-\alpha k}}{\pi} \int_0^\infty e^{-i \nu k} \frac{e^{-r T} \phi_{H93}(\nu - (\alpha + 1)i, T) \phi_{M76J}(\nu - (\alpha + 1)i, T)}{\alpha^2 + \alpha - \nu^2 + i (2\alpha + 1) \nu} \, d\nu
""")

<IPython.core.display.Math object>

In [ ]:
class SemiCompleteOptionPricingLibrary:
    _r: float
    _S0: float
    _K: float
    _T: float

    def __new__(self, r: float, S0: float, K: float, T: float) -> Self:
        self = super().__new__(self)
        self._r = r
        self._S0 = S0
        self._K = K
        self._T = T
        return self

    def heston_characteristic_function(self, u: Number, kappa_v: float,
                                         theta_v: float, sigma_v: float, rho: float, v0: float) -> complex:
        c1: float = kappa_v * theta_v
        c2: complex = -np.sqrt((rho * sigma_v * u * 1j - kappa_v) ** 2 - sigma_v**2 * (-u * 1j - u**2))
        c3: complex = (kappa_v - rho * sigma_v * u * 1j + c2) / (kappa_v - rho * sigma_v * u * 1j - c2)
        H1: complex = self._r * u * 1j * self._T + (c1 / sigma_v**2) * (
            (kappa_v - rho * sigma_v * u * 1j + c2) * self._T
            - 2 * np.log((1 - c3 * np.exp(c2 * self._T)) / (1 - c3))
        )
        H2: complex = ((kappa_v - rho * sigma_v * u * 1j + c2) / sigma_v**2) * (
            (1 - np.exp(c2 * self._T)) / (1 - c3 * np.exp(c2 * self._T))
        )
        return np.exp(H1 + H2 * v0)

    def heston_integration_function(self, u: float,
                                     kappa_v: float, theta_v: float, sigma_v: float, rho: float, v0: float) -> float:
        char_func: complex = self.heston_characteristic_function(u - 0.5 * 1j, kappa_v, theta_v, sigma_v, rho, v0)
        integrand: complex = np.exp(1j * u * np.log(self._S0 / self._K)) * char_func
        return (1 / (u**2 + 0.25)) * integrand.real

    def heston_call_value(self, kappa_v: float,
                          theta_v: float, sigma_v: float, rho: float, v0: float) -> float:
        int_value: float = quad(
            lambda u: self.heston_integration_function(u, kappa_v, theta_v, sigma_v, rho, v0),
            0,
            np.inf,
            limit=250,
        )[0]
        call_value: float = max(0, self._S0 - np.exp(-self._r * self._T) * np.sqrt(self._S0 * self._K) / np.pi * int_value)
        return call_value

    def merton_characteristic_function_jump(self, u: Number, lamb: float,
                                       mu: float, delta: float) -> complex:
        omega = -lamb * (np.exp(mu + 0.5 * delta**2) - 1)
        return np.exp((1j * u * omega + lamb * (np.exp(1j * u * mu - u**2 * delta**2 * 0.5) - 1)) * self._T)

    def merton_integration_function(self, u: float,
                                     sigma: float, lamb: float, mu: float, delta: float) -> float:
        char_func: complex = self.merton_characteristic_function(u - 0.5 * 1j, sigma, lamb, mu, delta)
        integrand: complex = np.exp(1j * u * np.log(self._S0 / self._K)) * char_func
        return (1 / (u**2 + 0.25)) * integrand.real

    def merton_call_value(self, sigma: float,
                          lamb: float, mu: float, delta: float) -> float:
        int_value: float = quad(
            lambda u: self.merton_integration_function(u, sigma, lamb, mu, delta),
            0,
            50,
            limit=250,
        )[0]
        call_value: float = max(0, self._S0 - np.exp(-self._r * self._T) * np.sqrt(self._S0 * self._K) / np.pi * int_value)
        return call_value

    def calibration_error_function(self, p0: NDArray[np.float_], options: pd.DataFrame,
                                   call_value_func: Callable[..., float]) -> float:
        squared_errors = []
        for _, row in options.iterrows():
            model_price: float = call_value_func(
                sigma=p0[0],
                lamb=p0[1],
                mu=p0[2],
                delta=p0[3]
            ) if call_value_func.__name__ == 'merton_call_value' else call_value_func(
                kappa_v=p0[0],
                theta_v=p0[1],
                sigma_v=p0[2],
                rho=p0[3],
                v0=p0[4]
            )
            squared_errors.append((model_price - row["Call"]) ** 2)
        rmse: float = np.sqrt(np.mean(squared_errors))
        return rmse

    def calibrate_merton(self, options: pd.DataFrame) -> NDArray[np.float_]:
        def error_function(p: NDArray[np.float_]) -> float:
            return self.calibration_error_function(p, options, self.merton_call_value)

        ranges = (
            slice(0.075, 0.201, 0.025),
            slice(0.10, 0.401, 0.1),
            slice(-0.5, 0.01, 0.1),
            slice(0.10, 0.301, 0.1),
        )
        p0: NDArray[np.float_] = brute(error_function, ranges, finish=None)
        opt: NDArray[np.float_] = fmin(
            error_function, p0, xtol=0.0001, ftol=0.0001, maxiter=550, maxfun=1050, disp=True
        )
        return opt

    def bates_characteristic_function(self, u: Number,
                                      kappa_v: float, theta_v: float, sigma_v: float,
                                      rho: float, v0: float, lamb: float, mu: float,
                                      delta: float) -> complex:
        heston_cf = self.heston_characteristic_function(u, kappa_v, theta_v, sigma_v, rho, v0)
        merton_jump_cf = self.merton_characteristic_function_jump(u, lamb, mu, delta)
        return heston_cf * merton_jump_cf

    def bates_integration_function(self, u: float,
                                   kappa_v: float, theta_v: float, sigma_v: float,
                                   rho: float, v0: float, lamb: float, mu: float,
                                   delta: float) -> float:
        char_func_value = self.bates_characteristic_function(
            u - 0.5j, kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta
        )
        integrand = np.exp(1j * u * np.log(self._S0 / self._K)) * char_func_value
        return (1 / (u**2 + 0.25)) * integrand.real

    def bates_call_value(self, kappa_v: float, theta_v: float, sigma_v: float,
                          rho: float, v0: float, lamb: float, mu: float,
                          delta: float) -> float:
        int_value = quad(
            lambda u: self.bates_integration_function(u, kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta),
            0,
            np.inf,
            limit=250,
        )[0]
        call_value = max(0, self._S0 - np.exp(-self._r * self._T) * np.sqrt(self._S0 * self._K) / np.pi * int_value)
        return call_value

    def bates_call_fft(self, kappa_v: float, theta_v: float, sigma_v: float,
                       rho: float, v0: float, lamb: float, mu: float,
                       delta: float) -> float:
        k = np.log(self._K / self._S0)
        g = 1
        N = int(g * 4096)
        eps = (g * 150) ** -1
        eta = 2 * np.pi / (N * eps)
        b_val = 0.5 * N * eps - k
        u = np.arange(1, N + 1, 1, dtype=float)
        vo = eta * (u - 1)

        if self._S0 >= 0.95 * self._K:
            alpha = 1.5
            v_arr = vo - (alpha + 1) * 1j
            modcharFunc = np.exp(-self._r * self._T) * (
                self.bates_characteristic_function(v_arr, kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta)
                / (alpha**2 + alpha - vo**2 + 1j * (2 * alpha + 1) * vo)
            )
        else:
            alpha = 1.1
            v_arr1 = (vo - 1j * alpha) - 1j
            modcharFunc1 = np.exp(-self._r * self._T) * (
                1 / (1 + 1j * (vo - 1j * alpha))
                - np.exp(self._r * self._T) / (1j * (vo - 1j * alpha))
                - self.bates_characteristic_function(v_arr1, kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta)
                / ((vo - 1j * alpha) ** 2 - 1j * (vo - 1j * alpha))
            )
            v_arr2 = (vo + 1j * alpha) - 1j
            modcharFunc2 = np.exp(-self._r * self._T) * (
                1 / (1 + 1j * (vo + 1j * alpha))
                - np.exp(self._r * self._T) / (1j * (vo + 1j * alpha))
                - self.bates_characteristic_function(v_arr2, kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta)
                / ((vo + 1j * alpha) ** 2 - 1j * (vo + 1j * alpha))
            )

        delt = np.zeros(N, dtype=np.float64)
        delt[0] = 1.0
        j = np.arange(1, N + 1, 1, dtype=np.int64)
        SimpsonW = (3 + (-1) ** j - delt) / 3

        if self._S0 >= 0.95 * self._K:
            FFTFunc = np.exp(1j * b_val * vo) * modcharFunc * eta * SimpsonW
            payoff = np.fft.fft(FFTFunc).real
            CallValueM = np.exp(-alpha * k) / np.pi * payoff
        else:
            FFTFunc = np.exp(1j * b_val * vo) * (modcharFunc1 - modcharFunc2) * 0.5 * eta * SimpsonW
            payoff = np.fft.fft(FFTFunc).real
            CallValueM = payoff / (np.sinh(alpha * k) * np.pi)

        pos = int((k + b_val) / eps)
        CallValue = CallValueM[pos] * self._S0
        return CallValue

In [ ]:
# General Parameters
S0 = 100         # Initial stock price
K = 100          # Strike price of the option
T = 1            # Time to expiration in years
r = 0.05         # Risk-free interest rate

# Heston '93 Parameters
kappa_v = 1.5    # Speed of mean reversion for volatility
theta_v = 0.02   # Long-term average volatility
sigma_v = 0.15   # Volatility of volatility (how much volatility fluctuates)
rho = 0.1        # Correlation between stock price and volatility
v0 = 0.01        # Initial variance (starting volatility squared)

# Merton '76 (Jump) Parameters
lamb = 0.25      # Jump intensity (average number of jumps per year)
mu = -0.2        # Average jump size (log-return of jumps)
delta = 0.1      # Standard deviation of jump size
sigma = np.sqrt(v0)  # For Bates, sigma from Merton is not used directly; set as sqrt of initial variance

# Create an instance of the OptionPricingLibrary (with Bates methods added)
pricing_lib = SemiCompleteOptionPricingLibrary(r, S0, K, T)

# Calculate the Bates call price via Lewis (2001) integration
bates_price_integration = pricing_lib.bates_call_value(
    kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta
)
print("Bates Call Price (Integration):", bates_price_integration)

# Calculate the Bates call price via FFT
bates_price_fft = pricing_lib.bates_call_fft(
    kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta
)
print("Bates Call Price (FFT):", bates_price_fft)


Bates Call Price (Integration): 8.904718863598816
Bates Call Price (FFT): 8.904718821081085


In [ ]:
heston_price_integration = pricing_lib.heston_call_value(kappa_v, theta_v, sigma_v, rho, v0)
print("Heston Call Price (Integration):", heston_price_integration)

Heston Call Price (Integration): 7.467062627562925
